In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test
from Models.basic_models import average_model, two_layer_basic

set_figsize(20, 15)

In [ ]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)


validation = TAVR_3_Frame("__valid", preproc="pixel")
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc="pixel")
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_basic(postprocessing="pixel")
loss_fn = batch_l2_loss()

In [3]:
for name, p in model.named_parameters():
    print(name, p.requires_grad)

conv_a1.weight True
conv_a1.bias True
conv_b1.weight True
conv_b1.bias True
final.weight True
final.bias True


In [4]:
learning_rate = 3e-2
momentum = 0.95
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg, nesterov=True)

In [ ]:
train(model, optimizer, train_loader, val_loader, loss_fn, device, 
         epochs=3, print_every=30, print_level=3, lr_decay=0.8)


****Epoch 0 Iteration 0, loss = 2.1082, corrected loss = 462.4782
Validation loss 453.7078 over 81 frames
conv_a1.weight,   	norm: 4.0584e+00, 	update norm: 1.4519e-01 	Update/norm: 3.5775e-02
conv_a1.bias,   	norm: 2.5633e-01, 	update norm: 2.5239e-02 	Update/norm: 9.8462e-02
conv_b1.weight,   	norm: 4.0513e+00, 	update norm: 1.4052e-01 	Update/norm: 3.4685e-02
conv_b1.bias,   	norm: 2.8699e-01, 	update norm: 2.6102e-02 	Update/norm: 9.0948e-02
final.weight,   	norm: 1.4505e+00, 	update norm: 1.6072e-01 	Update/norm: 1.1080e-01
final.bias,   	norm: 5.2234e-02, 	update norm: 5.1081e-02 	Update/norm: 9.7793e-01

Iter 0... ... ... 
Iter 10... ... ... 
Iter 20..

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
for p in model.parameters():
    print(p.norm())